# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error

In [19]:
df = pd.read_csv('previsao_de_renda.csv')

In [20]:
df = df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1).copy()


In [21]:
df.dropna(inplace=True)

In [22]:
df.educacao = df.educacao.map({'Superior completo':'SuperiorCompleto',
                            'Superior incompleto':'SuperiorIncompleto',
                            'Secundário':'Secundario',
                            'Primário':'Primario',
                           'Pós graduação':'PosGraduacao'})

In [23]:
df.tipo_renda = df.tipo_renda.map({'Assalariado':'Assalariado',
                            'Empresário':'Empresario',
                            'Servidor públicoo':'ServidorPublico',
                            'Bolsista':'Bolsista',
                           'Pensionista':'Pensionista'})

In [24]:
df.estado_civil = df.estado_civil.map({'Casado':'Casado',
                            'Solteiro':'Solteiro',
                            'União':'Uniao',
                            'Separadoa':'Separado',
                           'Viúvo':'Viuvo'})

In [25]:
df.tipo_residencia = df.tipo_residencia.map({'Casa':'Casa',
                            'Com os pais':'ComOsPais',
                            'Governamental':'Governamental',
                            'Aluguel':'Aluguel',
                           'Estúdio':'Estudio',
                            'Comunitário':'Comunitario'})

In [26]:
df = pd.get_dummies(df, columns=['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], drop_first=True)
df.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,posse_de_veiculo_True,posse_de_imovel_True,tipo_renda_Bolsista,tipo_renda_Empresario,...,educacao_SuperiorCompleto,educacao_SuperiorIncompleto,estado_civil_Solteiro,estado_civil_Uniao,estado_civil_Viuvo,tipo_residencia_Casa,tipo_residencia_ComOsPais,tipo_residencia_Comunitario,tipo_residencia_Estudio,tipo_residencia_Governamental
0,0,26,6.602740,1.0,8060.34,0,0,1,0,1,...,0,0,1,0,0,1,0,0,0,0
1,0,28,7.183562,2.0,1852.15,1,1,1,0,0,...,1,0,0,0,0,1,0,0,0,0
2,0,35,0.838356,2.0,2253.89,0,1,1,0,1,...,1,0,0,0,0,1,0,0,0,0
3,1,30,4.846575,3.0,6600.77,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
4,0,33,4.293151,1.0,6475.97,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1


#### 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [27]:
X = df.drop(['renda'], axis=1).copy()
y = df['renda'].copy().to_frame()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#### 2. Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o  𝑅2  na base de testes. Qual o melhor modelo?

In [29]:
df_teste = pd.concat([X_test, y_test], axis=1, sort=False).sort_index()
df_teste.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,posse_de_veiculo_True,posse_de_imovel_True,tipo_renda_Bolsista,tipo_renda_Empresario,tipo_renda_Pensionista,...,educacao_SuperiorIncompleto,estado_civil_Solteiro,estado_civil_Uniao,estado_civil_Viuvo,tipo_residencia_Casa,tipo_residencia_ComOsPais,tipo_residencia_Comunitario,tipo_residencia_Estudio,tipo_residencia_Governamental,renda
0,0,26,6.602740,1.0,0,0,1,0,1,0,...,0,1,0,0,1,0,0,0,0,8060.34
3,1,30,4.846575,3.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,6600.77
8,0,50,18.605479,2.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,3420.34
10,0,39,2.087671,2.0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,6402.41
16,0,53,8.545205,2.0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,3675.33


In [30]:
df_teste.columns

Index(['qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'sexo_M', 'posse_de_veiculo_True', 'posse_de_imovel_True',
       'tipo_renda_Bolsista', 'tipo_renda_Empresario',
       'tipo_renda_Pensionista', 'educacao_Primario', 'educacao_Secundario',
       'educacao_SuperiorCompleto', 'educacao_SuperiorIncompleto',
       'estado_civil_Solteiro', 'estado_civil_Uniao', 'estado_civil_Viuvo',
       'tipo_residencia_Casa', 'tipo_residencia_ComOsPais',
       'tipo_residencia_Comunitario', 'tipo_residencia_Estudio',
       'tipo_residencia_Governamental', 'renda'],
      dtype='object')

In [32]:
y, X = patsy.dmatrices('''renda ~ qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia +
       sexo_M + posse_de_veiculo_True + posse_de_imovel_True +
       tipo_renda_Bolsista + tipo_renda_Empresario +
       tipo_renda_Pensionista + educacao_Primario + 
       educacao_Secundario + educacao_SuperiorCompleto + 
       educacao_SuperiorIncompleto + estado_civil_Solteiro + 
       estado_civil_Uniao + estado_civil_Viuvo + tipo_residencia_Casa +
       tipo_residencia_ComOsPais + tipo_residencia_Comunitario +
       tipo_residencia_Estudio + tipo_residencia_Governamental'''
                       , data = df_teste)

In [34]:
X

DesignMatrix with shape (3107, 23)
  Columns:
    ['Intercept',
     'qtd_filhos',
     'idade',
     'tempo_emprego',
     'qt_pessoas_residencia',
     'sexo_M',
     'posse_de_veiculo_True',
     'posse_de_imovel_True',
     'tipo_renda_Bolsista',
     'tipo_renda_Empresario',
     'tipo_renda_Pensionista',
     'educacao_Primario',
     'educacao_Secundario',
     'educacao_SuperiorCompleto',
     'educacao_SuperiorIncompleto',
     'estado_civil_Solteiro',
     'estado_civil_Uniao',
     'estado_civil_Viuvo',
     'tipo_residencia_Casa',
     'tipo_residencia_ComOsPais',
     'tipo_residencia_Comunitario',
     'tipo_residencia_Estudio',
     'tipo_residencia_Governamental']
  Terms:
    'Intercept' (column 0)
    'qtd_filhos' (column 1)
    'idade' (column 2)
    'tempo_emprego' (column 3)
    'qt_pessoas_residencia' (column 4)
    'sexo_M' (column 5)
    'posse_de_veiculo_True' (column 6)
    'posse_de_imovel_True' (column 7)
    'tipo_renda_Bolsista' (column 8)
    'tipo_renda_

In [42]:
modelo_001 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.001
                         , alpha = 0.001)
modelo_001.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:21:26   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [44]:
modelo_005 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.001
                         , alpha = 0.005)
modelo_005.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:22:01   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [45]:
modelo_01 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.001
                         , alpha = 0.01)
modelo_01.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:22:09   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [46]:
modelo_05 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.001
                         , alpha = 0.05)
modelo_05.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:22:12   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [47]:
modelo_1 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.001
                         , alpha = 0.1)
modelo_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:22:21   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

#### Resposta
- Todos os modelos apresentam o mesmo valor de R² ajustado.

#### 3.Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [49]:
modelo_001 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.001)
modelo_001.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:22:55   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [51]:
modelo_005 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.005)
modelo_005.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:23:13   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [52]:
modelo_01 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.01)
modelo_01.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     62.00
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          8.64e-226
Time:                        16:23:21   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3085   BIC:                         6.333e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1559.0562   2454.367     -0.635      0.525   -6371.414    3253.302
qtd_filhos                      175.5541    496.286      0.354      0.724    -797.530    1148.639
idade                            48.0483     14.014      3.429      0.001      20.571      75.526
tempo_emprego                   505.2618     18.384     27.484      0.000     469.216     541.307
qt_pessoas_residencia            40.8451    478.285      0.085      0.932    -896.944     978.634
sexo_M                         5554.5948    257.600     21.563      0.000    5049.509    6059.680
posse_de_veiculo_True           598.9889    250.582      2.390      0.017     107.665    1090.313
posse_de_imovel_True            376.4199    242.871      1.550      0.121     -99.785     852.625
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0272    251.524      2.501      0.012     135.857    1122.198
tipo_renda_Pensionista        -2498.0321   4459.639     -0.560      0.575   -1.12e+04    6246.130
educacao_Primario             -1801.1912   2353.735     -0.765      0.444   -6416.238    2813.855
educacao_Secundario           -1518.6239   2001.513     -0.759      0.448   -5443.056    2405.809
educacao_SuperiorCompleto      -666.4749   2003.088     -0.333      0.739   -4593.997    3261.047
educacao_SuperiorIncompleto   -1800.6333   2067.566     -0.871      0.384   -5854.579    2253.312
estado_civil_Solteiro          -538.7171    557.988     -0.965      0.334   -1632.783     555.349
estado_civil_Uniao             -314.0026    454.473     -0.691      0.490   -1205.103     577.098
estado_civil_Viuvo             -276.0579    813.846     -0.339      0.734   -1871.793    1319.677
tipo_residencia_Casa            140.1149    882.609      0.159      0.874   -1590.446    1870.676
tipo_residencia_ComOsPais      -203.8174    994.387     -0.205      0.838   -2153.545    1745.910
tipo_residencia_Comunitario     -18.0908   1582.165     -0.011      0.991   -3120.294    3084.112
tipo_residencia_Estudio        3483.9757   1801.980      1.933      0.053     -49.227    7017.179
tipo_residencia_Governamental   852.4462   1100.792      0.774      0.439   -1305.913    3010.806
==============================================================================
Omnibus:                     2748.490   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142019.916
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [53]:
modelo_05 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.05)
modelo_05.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     64.97
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          1.06e-226
Time:                        16:23:24   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3086   BIC:                         6.332e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1564.3689   2409.595     -0.649      0.516   -6288.941    3160.203
qtd_filhos                      175.5586    496.205      0.354      0.724    -797.368    1148.485
idade                            48.0419     14.000      3.431      0.001      20.591      75.493
tempo_emprego                   505.2642     18.380     27.491      0.000     469.227     541.302
qt_pessoas_residencia            40.8503    478.207      0.085      0.932    -896.786     978.487
sexo_M                         5554.6870    257.432     21.577      0.000    5049.931    6059.443
posse_de_veiculo_True           598.9131    250.453      2.391      0.017     107.841    1089.985
posse_de_imovel_True            376.3578    242.771      1.550      0.121     -99.651     852.366
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0957    251.411      2.502      0.012     136.145    1122.046
tipo_renda_Pensionista        -2497.9563   4458.911     -0.560      0.575   -1.12e+04    6244.778
educacao_Primario             -1801.2002   2353.353     -0.765      0.444   -6415.498    2813.098
educacao_Secundario           -1518.5167   2001.166     -0.759      0.448   -5442.270    2405.236
educacao_SuperiorCompleto      -666.6103   2002.729     -0.333      0.739   -4593.426    3260.206
educacao_SuperiorIncompleto   -1800.5794   2067.226     -0.871      0.384   -5853.857    2252.698
estado_civil_Solteiro          -538.5219    557.637     -0.966      0.334   -1631.898     554.855
estado_civil_Uniao             -313.9377    454.364     -0.691      0.490   -1204.824     576.948
estado_civil_Viuvo             -276.1674    813.658     -0.339      0.734   -1871.533    1319.198
tipo_residencia_Casa            145.6385    738.552      0.197      0.844   -1302.465    1593.742
tipo_residencia_ComOsPais      -198.3404    871.273     -0.228      0.820   -1906.673    1509.993
tipo_residencia_Comunitario            0          0        nan        nan           0           0
tipo_residencia_Estudio        3489.5335   1734.905      2.011      0.044      87.847    6891.220
tipo_residencia_Governamental   857.9440    990.073      0.867      0.386   -1083.324    2799.212
==============================================================================
Omnibus:                     2748.492   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142020.122
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

In [54]:
modelo_1 = sm.OLS(y, X).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.1)
modelo_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     64.97
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          1.06e-226
Time:                        16:23:29   Log-Likelihood:                -31573.
No. Observations:                3107   AIC:                         6.319e+04
Df Residuals:                    3086   BIC:                         6.332e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1564.3689   2409.595     -0.649      0.516   -6288.941    3160.203
qtd_filhos                      175.5586    496.205      0.354      0.724    -797.368    1148.485
idade                            48.0419     14.000      3.431      0.001      20.591      75.493
tempo_emprego                   505.2642     18.380     27.491      0.000     469.227     541.302
qt_pessoas_residencia            40.8503    478.207      0.085      0.932    -896.786     978.487
sexo_M                         5554.6870    257.432     21.577      0.000    5049.931    6059.443
posse_de_veiculo_True           598.9131    250.453      2.391      0.017     107.841    1089.985
posse_de_imovel_True            376.3578    242.771      1.550      0.121     -99.651     852.366
tipo_renda_Bolsista                    0          0        nan        nan           0           0
tipo_renda_Empresario           629.0957    251.411      2.502      0.012     136.145    1122.046
tipo_renda_Pensionista        -2497.9563   4458.911     -0.560      0.575   -1.12e+04    6244.778
educacao_Primario             -1801.2002   2353.353     -0.765      0.444   -6415.498    2813.098
educacao_Secundario           -1518.5167   2001.166     -0.759      0.448   -5442.270    2405.236
educacao_SuperiorCompleto      -666.6103   2002.729     -0.333      0.739   -4593.426    3260.206
educacao_SuperiorIncompleto   -1800.5794   2067.226     -0.871      0.384   -5853.857    2252.698
estado_civil_Solteiro          -538.5219    557.637     -0.966      0.334   -1631.898     554.855
estado_civil_Uniao             -313.9377    454.364     -0.691      0.490   -1204.824     576.948
estado_civil_Viuvo             -276.1674    813.658     -0.339      0.734   -1871.533    1319.198
tipo_residencia_Casa            145.6385    738.552      0.197      0.844   -1302.465    1593.742
tipo_residencia_ComOsPais      -198.3404    871.273     -0.228      0.820   -1906.673    1509.993
tipo_residencia_Comunitario            0          0        nan        nan           0           0
tipo_residencia_Estudio        3489.5335   1734.905      2.011      0.044      87.847    6891.220
tipo_residencia_Governamental   857.9440    990.073      0.867      0.386   -1083.324    2799.212
==============================================================================
Omnibus:                     2748.492   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142020.122
Skew:                           3.994   Prob(JB):                         0.00
Kurtosis:                      35.144   Cond. No.                     1.84e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

#### Resposta

- Os 2 métodos chegaram no mesmo resultado

#### 4. Rode um modelo stepwise. Avalie o  𝑅2  na vase de testes. Qual o melhor resultado?

In [55]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [56]:
Xsw = X_test.copy()

In [57]:
ysw = y_test.copy()


In [58]:
variaveis = stepwise_selection(Xsw, ysw)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 8.47062e-122
#############
['tempo_emprego']
Add  sexo_M                         with p-value 8.19909e-117
#############
['tempo_emprego', 'sexo_M']
Add  educacao_SuperiorCompleto      with p-value 0.000226477
#############
['tempo_emprego', 'sexo_M', 'educacao_SuperiorCompleto']
Add  idade                          with p-value 0.000382414
#############
['tempo_emprego', 'sexo_M', 'educacao_SuperiorCompleto', 'idade']
Add  posse_de_veiculo_True          with p-value 0.00320998
#############
['tempo_emprego', 'sexo_M', 'educacao_SuperiorCompleto', 'idade', 'posse_de_veiculo_True']
Add  tipo_renda_Empresario          with p-value 0.0138968
#############
['tempo_emprego', 'sexo_M', 'educacao_SuperiorCompleto', 'idade', 'posse_de_veiculo_True', 'tipo_renda_Empresario']
Add  qt_pessoas_residencia          with p-value 0.0203004
#############
['tempo_emprego', 'sexo_M', 'educacao_SuperiorCompleto', 'idade', 'posse_de_veiculo_True', 'tipo_renda

In [59]:
variaveis

['tempo_emprego',
 'sexo_M',
 'educacao_SuperiorCompleto',
 'idade',
 'posse_de_veiculo_True',
 'tipo_renda_Empresario',
 'qt_pessoas_residencia',
 'tipo_residencia_Estudio']

In [61]:
df_teste_2 = pd.concat([Xsw, ysw], axis=1, sort=False).sort_index()
df_teste_2.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,posse_de_veiculo_True,posse_de_imovel_True,tipo_renda_Bolsista,tipo_renda_Empresario,tipo_renda_Pensionista,...,educacao_SuperiorIncompleto,estado_civil_Solteiro,estado_civil_Uniao,estado_civil_Viuvo,tipo_residencia_Casa,tipo_residencia_ComOsPais,tipo_residencia_Comunitario,tipo_residencia_Estudio,tipo_residencia_Governamental,renda
0,0,26,6.602740,1.0,0,0,1,0,1,0,...,0,1,0,0,1,0,0,0,0,8060.34
3,1,30,4.846575,3.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,6600.77
8,0,50,18.605479,2.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,3420.34
10,0,39,2.087671,2.0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,6402.41
16,0,53,8.545205,2.0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,3675.33


In [62]:
ysw, Xsw = patsy.dmatrices('''renda ~ tempo_emprego + sexo_M + educacao_SuperiorCompleto + idade + 
                        posse_de_veiculo_True + tipo_renda_Empresario + qt_pessoas_residencia + 
                        tipo_residencia_Estudio''', data=df_teste_2)

modelo_sw = sm.OLS(ysw, Xsw).fit()
modelo_sw.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.305
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     170.0
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          2.92e-238
Time:                        16:26:03   Log-Likelihood:                -31576.
No. Observations:                3107   AIC:                         6.317e+04
Df Residuals:                    3098   BIC:                         6.323e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                 -3578.5793    682.557     -5.243      0.000   -4916.889   -2240.270
tempo_emprego               504.4797     18.316     27.543      0.000     468.567     540.393
sexo_M                     5525.3393    255.020     21.666      0.000    5025.314    6025.365
educacao_SuperiorCompleto   853.2032    236.139      3.613      0.000     390.198    1316.209
idade                        55.6753     13.247      4.203      0.000      29.701      81.649
posse_de_veiculo_True       637.2925    247.788      2.572      0.010     151.447    1123.138
tipo_renda_Empresario       644.7148    250.050      2.578      0.010     154.435    1134.995
qt_pessoas_residencia       279.9288    119.202      2.348      0.019      46.206     513.651
tipo_residencia_Estudio    3316.9610   1579.504      2.100      0.036     219.980    6413.942
==============================================================================
Omnibus:                     2755.005   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           143156.424
Skew:                           4.008   Prob(JB):                         0.00
Kurtosis:                      35.273   Cond. No.                         593.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

#### Resposta:
 - Aparentemente o modelo step-wise apresentou como variáveis relevantes da regressão 8, ja os modelos por LASSO e RIDGE apontaram para 7 variáveis como significativamente relevantes para a regressão, a diferença ficou na variável qt_pessoa_residencia. Se compararmos o R² ajustado novamente o modelo step-wise tem um valor explicativo para a variabilidade dos dados melhor que os outros modelos, o mesmo acontece com o AIC, sendo respectivamente 0,303 e 63170.

#### 6. artindo dos modelos que você ajustou, tente melhorar o  𝑅2  na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis

In [63]:
df_teste.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,posse_de_veiculo_True,posse_de_imovel_True,tipo_renda_Bolsista,tipo_renda_Empresario,tipo_renda_Pensionista,...,educacao_SuperiorIncompleto,estado_civil_Solteiro,estado_civil_Uniao,estado_civil_Viuvo,tipo_residencia_Casa,tipo_residencia_ComOsPais,tipo_residencia_Comunitario,tipo_residencia_Estudio,tipo_residencia_Governamental,renda
0,0,26,6.602740,1.0,0,0,1,0,1,0,...,0,1,0,0,1,0,0,0,0,8060.34
3,1,30,4.846575,3.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,6600.77
8,0,50,18.605479,2.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,3420.34
10,0,39,2.087671,2.0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,6402.41
16,0,53,8.545205,2.0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,3675.33


In [64]:
ynew, Xnew = patsy.dmatrices('''np.log(renda) ~ sexo_M + posse_de_veiculo_True + idade + tempo_emprego +
                            tipo_renda_Empresario + tipo_residencia_Estudio''', data = df_teste)

In [65]:
modelo_lasso = sm.OLS(ynew, Xnew).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.1)
modelo_lasso.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     430.0
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          3.65e-295
Time:                        16:31:38   Log-Likelihood:                -3404.3
No. Observations:                3107   AIC:                             6819.
Df Residuals:                    3103   BIC:                             6849.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   7.3115      0.060    121.659      0.000       7.194       7.429
sexo_M                      0.8092      0.027     29.563      0.000       0.756       0.863
posse_de_veiculo_True            0          0        nan        nan           0           0
idade                       0.0039      0.001      2.665      0.008       0.001       0.007
tempo_emprego               0.0638      0.002     30.351      0.000       0.060       0.068
tipo_renda_Empresario            0          0        nan        nan           0           0
tipo_residencia_Estudio          0          0        nan        nan           0           0
==============================================================================
Omnibus:                        0.193   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.908   Jarque-Bera (JB):                0.203
Skew:                           0.019   Prob(JB):                        0.903
Kurtosis:                       2.990   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [66]:
modelo_ridge = sm.OLS(ynew, Xnew).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.001
                         , alpha = 0.1)
modelo_ridge.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     255.0
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          1.78e-300
Time:                        16:31:55   Log-Likelihood:                -3383.1
No. Observations:                3107   AIC:                             6782.
Df Residuals:                    3100   BIC:                             6830.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   7.2113      0.062    115.864      0.000       7.089       7.333
sexo_M                      0.7696      0.029     26.418      0.000       0.712       0.827
posse_de_veiculo_True       0.1091      0.028      3.875      0.000       0.054       0.164
idade                       0.0045      0.001      3.046      0.002       0.002       0.007
tempo_emprego               0.0648      0.002     30.902      0.000       0.061       0.069
tipo_renda_Empresario       0.1214      0.029      4.256      0.000       0.065       0.177
tipo_residencia_Estudio     0.4879      0.181      2.699      0.007       0.133       0.842
==============================================================================
Omnibus:                        0.105   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.949   Jarque-Bera (JB):                0.114
Skew:                           0.014   Prob(JB):                        0.944
Kurtosis:                       2.990   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### Resposta

- O modelo ridge utilizando o log(renda) como variável resposta e removendo as variáveis nao significativas melhorou seu R² ajustado para 0,364 ou seja consegue explicar por 36% da variabilidade dos dados e o AIC caiu de 63170 para 6782 uma melhora significativa para o modelo.

#### 7. Ajuste uma árvore de regressão e veja se consegue um  𝑅2  melhor com ela.

In [67]:
arvore = DecisionTreeRegressor()

In [68]:
arvore.fit(X_train, y_train);

In [72]:
r2_1 = arvore.score(X_test, y_test)
print(f'O R² da árvore com profundidade {arvore.get_depth()} na base teste é: {round(r2_1, 2)}')

mse_1 = mean_squared_error(y_test , arvore.predict(X_test))
print(f'O MSE da árvore na base de teste é: {round(mse_1, 2)}')

O R² da árvore com profundidade 30 na base teste é: 0.18
O MSE da árvore na base de teste é: 46466803.56


In [79]:
arvore_2 = DecisionTreeRegressor(max_depth=3)
arvore_2.fit(X_train, y_train);

In [80]:
r2_1 = arvore_2.score(X_test, y_test)
print(f'O R² da árvore com profundidade {arvore_2.get_depth()} na base teste é: {round(r2_1, 2)}')

mse_1 = mean_squared_error(y_test , arvore_2.predict(X_test))
print(f'O MSE da árvore na base de teste é: {round(mse_1, 2)}')

O R² da árvore com profundidade 3 na base teste é: 0.31
O MSE da árvore na base de teste é: 38871710.83


#### Resposta

- Não foi possível ajustar uma arvore de regressão melhor que os modelos ajustados anteriomente. Sendo que o melhor modelo ajustado foi o RIDGE com a variável resposta log(renda) e as variáveis explicativa sexo_M, posse_de_veiculo_True, idade,  tempo_emprego, tipo_renda_Empresario, tipo_residencia_Estudio. Com R² ajustado de 0,364 e AIC 6782.